In [1]:
#import sys
# !{sys.executable} -m pip install pymysql
# !{sys.executable} -m pip install ratelimiter
# !{sys.executable} -m pip install sqlalchemy

import pandas as pd
import requests
from ratelimiter import RateLimiter
from Location import Location, LocationTools


In [5]:
ufos = pd.read_csv(r'/Users/kellan/Downloads/ufo-reports-master/csv-data/ufo-scrubbed-geocoded-time-standardized.csv')
ufos_canada = ufos[ufos['Country'] == 'ca']

ufos_canada.head()

,Date,City,State,Country,Type,Seconds,Duration,Description,Date added,Lat,Lon
61,10/10/1994 23:00,toronto (greater toronto area) (canada),on,ca,sphere,3600,~1 hour,Large rusty sphere,7/3/2013,43.666667,-79.416667
80,10/10/1998 22:30,st. john&#39s (canada),nf,ca,egg,7200,2 hours,Started off as 3 points of intense yellow ligh...,12/2/2000,47.55,-52.666667
95,10/10/2000 07:30,victoria (canada),bc,ca,cylinder,30,30seconds,Smooth Shiny Cylinder,12/2/2000,46.216667,-63.483333
104,10/10/2001 20:10,vancouver (canada),bc,ca,other,300,+5 minutes,I observed an green object significantly above...,5/12/2011,49.25,-123.133333
114,10/10/2002 19:45,victoria (canada),bc,ca,unknown,120,2 minutes approx,bright white light with black outline around i...,10/15/2002,46.216667,-63.483333


As we can see, there are some problems with the location data. Let's delete the columns and recalculate

In [6]:
ufos_canada.State.unique()
ufos_canada.drop(columns=['State', 'City', 'Country', 'Date added'], inplace =True)

Now, let's call the api using Location, and calculate our columns. The reverse geocoding is done through the locationiq.com API

In [ ]:

location_info = Location.Location()

with open("token.txt") as f:
    private_token = f.read()

location_columns = ['full_address', 'neighbourhood', 'city', 'province', 'country_code', 'country']
for col in location_columns:
    ufos_canada[col] = ''

ufos_canada['location_data_raw'] = ufos_canada.apply(lambda x: Location.make_request(x['Lat'], 
                                                                                     x['Lon'], 
                                                                                     private_token).return_raw(), axis=1)

location_columns = ['full_address', 'neighbourhood', 'city', 'province', 'country_code', 'country']
ufos_canada[location_columns] = ufos_canada.apply(lambda x: Location.get_all(x['location_data_raw']), axis=1)
ufos_canada.reset_index(inplace=True, drop=True)
ufos_canada.columns = ufos_canada.columns.str.title().str.strip()

ufos_canada.to_csv('canada_ufos_corrected.csv', index=False)

In [ ]:
ufos_canada

Next, we now will write the full dataset to a table, before breaking up the data set into separate provinces